In [1]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

In [2]:
%%sql
CREATE TABLE Projects
(workorder_id CHAR(5) NOT NULL,
 step_nbr INTEGER NOT NULL
    CHECK (step_nbr BETWEEN 0 AND 1000),
 step_status CHAR(1) NOT NULL
    CHECK (step_status IN ('C', 'W')), -- Cは完了、Wは待機
    PRIMARY KEY (workorder_id, step_nbr));

INSERT INTO Projects VALUES('AA100', 0, 'C' );
INSERT INTO Projects VALUES('AA100', 1, 'W' );
INSERT INTO Projects VALUES('AA100', 2, 'W' );
INSERT INTO Projects VALUES('AA200', 0, 'W' );
INSERT INTO Projects VALUES('AA200', 1, 'W' );
INSERT INTO Projects VALUES('AA300', 0, 'C' );
INSERT INTO Projects VALUES('AA300', 1, 'C' );

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

## やりたいこと
+ group内の要素のうち、複数の条件を満たすgroup名を取得する

In [3]:
%%sql
select workorder_id
from Projects
group by workorder_id
having count(*) = count(case when step_nbr = 0 and step_status = 'C' then 1 else null end) +
    count(case when step_nbr <> 0 and step_status = 'W' then 1 else null end)

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


workorder_id
AA100


In [6]:
%%sql
select workorder_id
from Projects
group by workorder_id
having sum(case when (
        step_nbr = 0 and step_status = 'C'
    ) or (
        step_nbr <> 0 and step_status = 'W'
    ) then 1 else 0
end ) = count(*)

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


workorder_id
AA100


In [4]:
%%sql
select workorder_id
from Projects
where step_status = 'C'
group by workorder_id
having sum(step_nbr) = 0

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


workorder_id
AA100


In [5]:
%%sql
select workorder_id
from (
    select min(step_nbr) as min, step_status, workorder_id, count(*) as cnt
    from Projects as p1
    group by workorder_id, step_status
) as tmp
where (step_status = 'C' and min = 0 and cnt = 1)

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


workorder_id
AA100


In [7]:
%%sql
-- not existsを使う
select workorder_id
from Projects as p1
where step_nbr = 0
and step_status = 'C'
and not exists (
    select *
    from Projects as p2
    where p1.workorder_id = p2.workorder_id and
    p1.step_nbr <> p2.step_nbr and
    p2.step_status <> 'W'
)

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


workorder_id
AA100


In [8]:
%%sql
-- not existsを使う(where句に要件を全てまとめる)
select distinct workorder_id
from Projects as p1
where not exists (
    select *
    from Projects as p2
    where p1.workorder_id = p2.workorder_id and
    p2.step_status <> (
        case when step_nbr = 0 then 'C' else 'W' end
    )
)

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


workorder_id
AA100
